In [26]:
from konlpy.tag import Mecab
from konlpy.corpus import kolaw
import pygtrie
import numpy as np
from numpy import random

In [11]:
fids = kolaw.fileids()
fobj = kolaw.open(fids[0])
text = fobj.read()
fobj.close()

In [14]:
mecab = Mecab()
morphs = mecab.morphs(text)

In [22]:
grams = pygtrie.StringTrie()
vocab = []

In [23]:
for i in range(len(morphs)):
    if morphs[i] not in vocab:
        vocab.append(morphs[i])
    for n in range(1, 6):
        if n - 1 <= i:
            ngram = '/'.join(morphs[i-n+1:i+1])
            try:
                grams[ngram] += 1
            except KeyError:
                grams[ngram] = 1

In [35]:
for n in range(2, 6):
    print('========== ' + ('bi' if n == 2 else ('tri' if n == 3 else f'{n}-')) + 'gram ==========')
    print()
    
    ngram = [''] * n
    sentence_len = 0
    history_len = 0
    
    while ngram[-1] != '.':
        if 200 <= sentence_len:
            print(' <<< The sentence being generated has exceeded 200 tokens. Sentence finishing failed. >>>')
            break
        
        ngram[:-1] = ngram[1:]
        
        p = np.zeros(len(vocab))
        
        if history_len == 0:
            for i in range(len(vocab)):
                try:
                    p[i] = grams['.' + '/' + vocab[i]]
                except KeyError:
                    pass
        else:
            history = '/'.join(ngram[-history_len-1:-1])
            for i in range(len(vocab)):
                try:
                    p[i] = grams[history + '/' + vocab[i]]
                except KeyError:
                    pass
        
        p /= p.sum()
        
        ngram[-1] = vocab[random.choice(len(vocab), p=p)]
        print(ngram[-1])
        sentence_len += 1
        if history_len < n - 1:
            history_len += 1
    
    print()

========== bigram ==========

"
제
86
조
①
헌법
에
관한
감찰
을
가지
는
죄
중
에
따라
직무
집행
하
지
아니하
는
법률
로
정한다
.

========== trigram ==========

②
이
헌법
시행
당시
의
대법원장
과
대
법원
은
법률
로
정한다
.

========== 4-gram ==========

②
저작
자
·
발명가
·
과학
기술자
와
예술가
의
권리
는
법률
로써
보호
한다
.

========== 5-gram ==========

제
92
조
①
평화통일
정책
의
수립
에
관하
여
대통령
의
명
을
받
아
행정각부
를
통할
한다
.

